# Init

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip -q "/content/drive/MyDrive/BOAZ_/total.zip" -d "/content/drive/MyDrive/BOAZ_"

In [ ]:
!zip -r "/content/drive/MyDrive/BOAZ_/total_prc.zip" "/content/drive/MyDrive/BOAZ_/total_processed"

# 데이터 처리

## Gemini test용 데이터 자르기

In [ ]:
pip install Pillow

In [ ]:
from PIL import Image
import os
import glob

# --- 설정 변수 ---
INPUT_DIR = "/content/drive/MyDrive/BOAZ_/total"          # 이미지가 들어있는 폴더 이름
OUTPUT_DIR = "/content/drive/MyDrive/BOAZ_/total_processed"   # 분할된 이미지를 저장할 폴더 이름
SPLIT_COUNT = 13                    # 이미지 분할 개수 (13등분)

def split_image_into_N_vertical_parts(image_path, output_dir, N):
    """
    단일 이미지를 세로 방향으로 N개의 조각(높이 분할)으로 분할하여 저장합니다.
    """
    try:
        # 1. 이미지 로드
        img = Image.open(image_path)

        # 2. 이미지 정보
        width, height = img.size

        # 3. 각 조각의 높이 계산 (정수 나눗셈)
        part_height = height // N

        # 4. 파일 이름 및 확장자 분리
        base_name, ext = os.path.splitext(os.path.basename(image_path))

        print(f"\n[처리 중] 파일: {os.path.basename(image_path)} (크기: {width}x{height})")
        print(f"  -> 각 조각 높이: 약 {part_height} 픽셀")

        # 5. N개의 조각 자르기 및 저장
        for i in range(N):
            # i번째 조각의 위쪽 경계 (Top)
            top = i * part_height

            # 아래쪽 경계 (Bottom): 마지막 조각은 나머지 높이를 모두 포함하여 정확한 분할 보장
            if i == N - 1:
                bottom = height
            else:
                bottom = (i + 1) * part_height

            # 이미지 자르기 (crop): (left, top, right, bottom)
            cropped_img = img.crop((0, top, width, bottom))

            # 새 파일 이름 설정 (예: image_split_v12_01.png)
            output_filename = os.path.join(output_dir, f"{base_name}_split_v{N}_{i+1:02d}{ext}")

            # 파일 저장
            cropped_img.save(output_filename)
            print(f"  -> 저장 완료: {os.path.basename(output_filename)}")

    except Exception as e:
        print(f"  ❌ 오류 발생: {image_path} 처리 중 오류 - {e}")

def process_images_in_target_directory():
    """
    INPUT_DIR에 지정된 폴더의 모든 이미지 파일을 찾아 세로로 SPLIT_COUNT 등분합니다.
    """

    # 처리할 이미지 파일 확장자 목록 (소문자)
    image_extensions = ('*.png', '*.jpg', '*.jpeg')

    # 출력 폴더 생성
    output_path = OUTPUT_DIR # os.path.join(os.getcwd(), OUTPUT_DIR) # Modified to use absolute path directly
    if not os.path.exists(output_path):
        os.makedirs(output_path)
        print(f"출력 폴더 생성: {output_path}")

    print(f"\n--- 이미지 세로 {SPLIT_COUNT}등분 일괄 처리 시작 (대상 폴더: {INPUT_DIR}) ---")

    found_count = 0

    # INPUT_DIR 폴더에서 이미지 파일을 검색
    for ext in image_extensions:
        search_path = os.path.join(INPUT_DIR, ext)
        for image_path in glob.glob(search_path):
            split_image_into_N_vertical_parts(image_path, output_path, SPLIT_COUNT)
            found_count += 1

    if found_count == 0:
        print(f"\n⚠️ 오류: 폴더 '{INPUT_DIR}'에서 처리할 이미지 파일(png, jpg, jpeg)을 찾을 수 없습니다.")
        print("   - 폴더 이름이 맞는지, 이미지가 폴더 안에 있는지 확인해주세요.")
    else:
        print(f"\n✅ 총 {found_count}개의 이미지 파일 처리가 완료되었습니다.")

# --- 메인 함수 실행 ---
process_images_in_target_directory()

# 코드 구현

## CLIP 임베딩 변환 및 저장, 테스트 (Retreival)

In [ ]:
# 필요한 패키지 설치
!pip install -q transformers torch pillow chromadb

In [ ]:
import os
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import chromadb
from chromadb.utils import embedding_functions

# ==========================================
# 1. 설정 및 모델 로드
# ==========================================

# GPU 사용 가능 여부 확인
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# CLIP 모델 로드 (OpenAI의 기본 모델 사용)
model_id = "openai/clip-vit-base-patch32"
model = CLIPModel.from_pretrained(model_id).to(device)
processor = CLIPProcessor.from_pretrained(model_id)

# ChromaDB 클라이언트 초기화 (메모리 모드 - Colab 세션 끝나면 사라짐)
# 데이터를 영구 저장하려면: chromadb.PersistentClient(path="./my_db") 사용
client = chromadb.Client()

# 컬렉션 생성 (이미 있다면 삭제 후 새로 생성)
collection_name = "webtoon_rag"
try:
    client.delete_collection(name=collection_name)
except:
    pass

# CLIP은 코사인 유사도를 사용하므로 distance metric을 'cosine'으로 설정
collection = client.create_collection(
    name=collection_name,
    metadata={"hnsw:space": "cosine"}
)

print("✅ 모델 로드 및 DB 초기화 완료")

# ==========================================
# 2. 임베딩 함수 정의
# ==========================================

def get_image_embedding(image_path):
    try:
        image = Image.open(image_path)
        # CLIP 프로세서를 통해 전처리 (이미지 -> 텐서)
        inputs = processor(images=image, return_tensors="pt").to(device)

        # 임베딩 추출
        with torch.no_grad():
            image_features = model.get_image_features(**inputs)

        # 정규화 (Normalization) - 코사인 유사도 성능 향상
        image_features = image_features / image_features.norm(p=2, dim=-1, keepdim=True)

        # 리스트 형태로 변환하여 반환
        return image_features.cpu().numpy()[0].tolist()

    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None

# ==========================================
# 3. 이미지 폴더 순회 및 DB 저장
# ==========================================

# ⚠️ [중요] 분할된 이미지가 있는 폴더 경로를 지정하세요!
# 예: Colab에 'images' 폴더를 만들고 업로드했다면 "./images"
IMAGE_FOLDER_PATH = "/content/drive/MyDrive/BOAZ_/total_processed"

# 폴더가 없으면 에러 방지를 위해 생성 (테스트용)
if not os.path.exists(IMAGE_FOLDER_PATH):
    os.makedirs(IMAGE_FOLDER_PATH)
    print(f"⚠️ 경고: '{IMAGE_FOLDER_PATH}' 폴더가 없어 생성했습니다. 이미지를 넣어주세요.")

# 이미지 파일 리스트 가져오기
image_files = [f for f in os.listdir(IMAGE_FOLDER_PATH) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

print(f"총 {len(image_files)}개의 이미지를 처리합니다...")

ids = []
embeddings = []
metadatas = []

for idx, file_name in enumerate(image_files):
    file_path = os.path.join(IMAGE_FOLDER_PATH, file_name)

    # 2단계: 임베딩 추출
    embedding = get_image_embedding(file_path)

    if embedding is not None:
        ids.append(file_name) # ID는 파일명으로 설정
        embeddings.append(embedding)
        # 메타데이터에 원본 경로 저장 (나중에 이미지를 불러올 때 필요)
        metadatas.append({"source": "webtoon", "file_path": file_path})

    if (idx + 1) % 10 == 0:
        print(f"{idx + 1}개 처리 완료...")

# 3단계: ChromaDB에 저장
if ids:
    collection.add(
        ids=ids,
        embeddings=embeddings,
        metadatas=metadatas
    )
    print(f"\n🎉 성공! 총 {len(ids)}개의 웹툰 컷이 Vector DB에 저장되었습니다.")
else:
    print("\n❌ 저장할 이미지가 없습니다. 폴더 경로를 확인해주세요.")

In [ ]:
# ==========================================
# 4. 텍스트로 이미지 검색 (Retrieval 테스트)
# ==========================================

def search_webtoon(query_text, top_k=3):
    # 1. 텍스트 질문을 임베딩으로 변환
    inputs = processor(text=[query_text], return_tensors="pt", padding=True).to(device)
    with torch.no_grad():
        text_features = model.get_text_features(**inputs)

    # 정규화
    text_features = text_features / text_features.norm(p=2, dim=-1, keepdim=True)
    query_embedding = text_features.cpu().numpy()[0].tolist()

    # 2. Vector DB에서 유사한 이미지 검색
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )

    return results

# 테스트 질문
query = "Pink shirts, short haired woman"

print(f"🔍 질문: '{query}'")
results = search_webtoon(query)

# 결과 출력
print("\n[검색 결과]")
for i in range(len(results['ids'][0])):
    file_name = results['ids'][0][i]
    distance = results['distances'][0][i] # 거리가 가까울수록(0에 가까울수록) 유사함 (cosine distance)
    print(f"{i+1}순위: {file_name} (거리: {distance:.4f})")

    # Colab에서 이미지 바로 보기
    from IPython.display import display
    img_path = results['metadatas'][0][i]['file_path']
    display(Image.open(img_path).resize((200, 200))) # 보기 좋게 리사이징
    print("-" * 30)

## 다국어 CLIP ver code

In [ ]:
!pip install sentence_transformers chromadb

In [ ]:
import os
from sentence_transformers import SentenceTransformer
import chromadb
from PIL import Image
from IPython.display import display
import numpy as np

# ==========================================
# 1. 모델 및 DB 설정
# ==========================================
print("🚀 모델 로드 중...")
model = SentenceTransformer('clip-ViT-B-32-multilingual-v1')

client = chromadb.Client()
collection_name = "webtoon_kor_fixed"

# 기존 컬렉션 초기화
try:
    client.delete_collection(name=collection_name)
except:
    pass
collection = client.create_collection(name=collection_name)

# ==========================================
# 2. 이미지 배치 처리 (Batch Processing) - 핵심! ⭐
# ==========================================
IMAGE_FOLDER_PATH = "/content/drive/MyDrive/BOAZ_/total_processed"

if not os.path.exists(IMAGE_FOLDER_PATH):
    print(f"❌ 경로 오류: '{IMAGE_FOLDER_PATH}' 폴더가 없습니다.")
else:
    image_files = [f for f in os.listdir(IMAGE_FOLDER_PATH) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    total_files = len(image_files)
    print(f"📂 총 {total_files}개의 이미지를 처리합니다.")

    # 배치 설정 (한 번에 32장씩 처리)
    BATCH_SIZE = 32

    # 임시 저장소
    batch_images = []
    batch_ids = []
    batch_metadatas = []

    for idx, file_name in enumerate(image_files):
        file_path = os.path.join(IMAGE_FOLDER_PATH, file_name)

        try:
            # 1. 이미지 로드 및 RGB 변환 (PNG 투명도 문제 방지)
            img = Image.open(file_path).convert('RGB')

            batch_images.append(img)
            batch_ids.append(file_name)
            batch_metadatas.append({"file_path": file_path})

            # 2. 배치가 꽉 찼거나 마지막 이미지일 때 -> 인코딩 실행
            if len(batch_images) >= BATCH_SIZE or idx == total_files - 1:

                # [핵심 수정] 리스트 형태(batch_images)로 넘겨줍니다.
                embeddings = model.encode(batch_images).tolist()

                # DB 저장
                collection.add(
                    ids=batch_ids,
                    embeddings=embeddings,
                    metadatas=batch_metadatas
                )

                print(f"✅ {idx + 1}/{total_files} 저장 완료")

                # 배치 초기화
                batch_images = []
                batch_ids = []
                batch_metadatas = []

        except Exception as e:
            print(f"⚠️ Error processing {file_name}: {e}")
            # 에러난 이미지가 있으면 배치에서 제외하고 계속 진행
            if len(batch_images) > len(batch_ids):
                batch_images.pop()

    print("\n🎉 모든 이미지 저장 완료!")

# ==========================================
# 3. 검색 테스트
# ==========================================
def search_korean(query_text):
    print(f"\n🔍 검색어: '{query_text}'")
    # 텍스트도 리스트로 감싸주는 것이 안전합니다.
    query_emb = model.encode([query_text]).tolist()

    results = collection.query(
        query_embeddings=query_emb,
        n_results=3
    )

    for i in range(len(results['ids'][0])):
        file_name = results['ids'][0][i]
        distance = results['distances'][0][i]
        path = results['metadatas'][0][i]['file_path']

        print(f"[{i+1}위] {file_name} (거리: {distance:.4f})")
        try:
            display(Image.open(path).resize((200, 200)))
        except:
            print("이미지를 불러올 수 없습니다.")
        print("-" * 30)

# 바로 테스트 해보세요!
search_korean("핑크 셔츠 입은 여자")
search_korean("넥타이 맨 남자")

# text OCR 을 위한 이미지 전처리


In [ ]:
import cv2
import numpy as np
import os

def extract_text_regions(image_path, save_dir, padding=20):
    """
    웹툰 이미지에서 텍스트(말풍선) 구역만 감지하여 잘라내는 함수
    Args:
        image_path: 원본 이미지 경로
        save_dir: 저장할 폴더
        padding: 텍스트 박스 주변에 줄 여백 (px)
    """

    # 1. 이미지 로드
    img_array = np.fromfile(image_path, np.uint8)
    img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

    if img is None:
        return

    file_name = os.path.splitext(os.path.basename(image_path))[0]

    # 2. 그레이스케일 변환
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 3. 엣지 검출 및 이진화 (글씨를 하얗게, 배경을 검게)
    # 적응형 이진화(Adaptive Threshold)를 쓰면 그림자 진 말풍선도 잘 찾습니다.
    binary = cv2.adaptiveThreshold(
        gray, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV,
        11, 5
    )

    # 4. 모폴로지 연산 (팽창) - 핵심! ⭐
    # 글자들을 뚱뚱하게 만들어서 한 덩어리로 뭉칩니다.
    # 커널 사이즈 (15, 3)은 가로로 긴 텍스트(문장)를 잘 뭉치게 합니다.
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 5))
    dilated = cv2.dilate(binary, kernel, iterations=3)

    # 5. 윤곽선(덩어리) 찾기
    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    count = 0
    img_height, img_width = img.shape[:2]

    # 윤곽선 순서를 위에서 아래로 정렬 (스크롤 순서 유지)
    bounding_boxes = [cv2.boundingRect(c) for c in contours]
    bounding_boxes.sort(key=lambda x: x[1]) # y좌표 기준 정렬

    for (x, y, w, h) in bounding_boxes:

        # 6. 필터링 (너무 작거나 이상한 비율은 버림)
        # 예: 너비가 너무 좁거나(세로선), 높이가 너무 낮거나(점), 화면 전체거나
        if w < 20 or h < 20: continue # 너무 작은 점
        if w > img_width * 0.9 and h > img_height * 0.9: continue # 이미지 전체

        # 비율 필터 (말풍선은 보통 덩어리 형태)
        aspect_ratio = w / float(h)
        if aspect_ratio < 0.2 or aspect_ratio > 10: continue

        # 7. 패딩을 줘서 넉넉하게 자르기 (글자 잘림 방지)
        x1 = max(0, x - padding)
        y1 = max(0, y - padding)
        x2 = min(img_width, x + w + padding)
        y2 = min(img_height, y + h + padding)

        roi = img[y1:y2, x1:x2]

        # 저장
        save_path = os.path.join(save_dir, f"{file_name}_text_{count:03d}.jpg")
        is_success, im_buf_arr = cv2.imencode(".jpg", roi)
        if is_success:
            im_buf_arr.tofile(save_path)
            count += 1

    print(f"✅ {file_name}: 텍스트 구역 {count}개 추출 완료")

# ==========================================
# 실행
# ==========================================
INPUT_DIR = "/content/drive/MyDrive/BOAZ_/raw_webtoons"
OUTPUT_DIR = "/content/drive/MyDrive/BOAZ_/ocr_crops"

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

files = [f for f in os.listdir(INPUT_DIR) if f.lower().endswith(('.jpg', '.png'))]

for f in files:
    path = os.path.join(INPUT_DIR, f)
    extract_text_regions(path, OUTPUT_DIR, padding=30)

# JoyCaption test

In [ ]:
# 1. 기존 라이브러리 강제 삭제
!pip uninstall -y transformers accelerate bitsandbytes

# 2. 호환성이 검증된 버전으로 재설치
!pip install -q -U transformers==4.46.3 accelerate bitsandbytes "pillow<11.0.0"

In [ ]:
import torch
import os
import gc
from transformers import AutoProcessor, LlavaForConditionalGeneration
from PIL import Image

# 1. 환경 정리
gc.collect()
torch.cuda.empty_cache()

model_id = "fancyfeast/llama-joycaption-beta-one-hf-llava"

# 2. 8-bit 로드 (T4 GPU에서 가장 호환성이 높음)
print("🚀 [System] 모델 로딩 및 타입 최적화 시작...")
processor = AutoProcessor.from_pretrained(model_id)
model = LlavaForConditionalGeneration.from_pretrained(
    model_id,
    load_in_8bit=True,
    device_map={"": 0},
    trust_remote_code=True
)

# 모델의 실제 연산 타입을 변수에 저장 (핵심)
target_dtype = model.dtype
print(f"✅ 모델 로드 완료! 연산 정밀도: {target_dtype}")

# 3. 캡션 생성 함수 (강제 캐스팅 로직 포함)
def generate_webtoon_caption(image_path):
    raw_image = Image.open(image_path).convert("RGB")

    prompt = "Describe this webtoon panel in detail. Focus on the character's facial expression, the emotional vibe of the scene, and any notable artistic styles."
    formatted_prompt = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n<image>\n{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

    # 텐서 생성
    inputs = processor(text=[formatted_prompt], images=[raw_image], return_tensors="pt").to("cuda")

    # [핵심] 모든 입력 텐서를 모델의 가중치 타입(target_dtype)으로 강제 변환
    # FloatTensor(32) -> HalfTensor(16) 변환을 수행하여 충돌 방지
    for k, v in inputs.items():
        if isinstance(v, torch.Tensor) and torch.is_floating_point(v):
            inputs[k] = v.to(target_dtype)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=300,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
            eos_token_id=processor.tokenizer.eos_token_id
        )

    decoded = processor.decode(output[0], skip_special_tokens=True)
    return decoded.split("assistant")[-1].strip()

# 4. 실행
target_path = "/content/drive/MyDrive/BOAZ_/total_processed/screencapture-comic-naver-webtoon-detail-2025-11-29-12_37_14-4_split_v13_01.png"

if os.path.exists(target_path):
    print(f"\n🔍 분석 시작: {os.path.basename(target_path)}")
    try:
        # GPU 캐시 비우고 실행
        torch.cuda.empty_cache()
        result = generate_webtoon_caption(target_path)
        print("\n" + "="*50)
        print(f"[JoyCaption 결과]\n{result}")
        print("="*50)
    except Exception as e:
        print(f"❌ 최종 시도 실패: {e}")
        # 실패 시 타입 정보 출력 (디버깅용)
        print(f"DEBUG: Input dtype: {inputs['pixel_values'].dtype if 'inputs' in locals() else 'N/A'}")
        print(f"DEBUG: Model dtype: {model.dtype}")

In [ ]:
image_path = "/content/drive/MyDrive/BOAZ_/total_processed/screencapture-comic-naver-webtoon-detail-2025-11-29-12_37_14-4_split_v13_01.png"

if os.path.exists(image_path):
    result = generate_caption(image_path, "Describe this webtoon scene in detail, focusing on expressions and atmosphere.")
    print("\n[캡션 결과]\n", result)
else:
    print("경로를 확인해주세요.")